In [7]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import tensorflow as tf
import argparse
import numpy as np
import plotly.graph_objects as go

tf.random.set_seed(0)

In [69]:
np.load('dataset/data_N365_5.npz')['paras']

array([[0.5 , 0.5 , 2.  , 0.97]])

In [4]:
def MLP(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model
def RNNmodel(input_dim, output_dim):
    model = Sequential()
    model.add(LSTM(units=50,return_sequences=True,input_shape=(input_dim, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=output_dim))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model


In [70]:
df_dp = np.load(f"dataset/data_N365_2.npz")
df_dp['paras']

array([[0.5 , 0.5 , 2.  , 0.81]])

In [62]:
df_dp = np.load(f"dataset/data_N365_0.npz")
df_price = df_dp["price"]

T = 24
N_train = 300
P1 = df_dp["p"].max()
P2 = df_dp["d"].max()
d = df_dp["d"]
p = df_dp["p"]
price_tensor = df_price[0:N_train]
d_tensor = d[0:N_train]
p_tensor = p[0:N_train]

# build model
model = RNNmodel(T,T)

df_y= p-d
test_price = df_price[:10]
test_dr = df_y[:10]*100
train_price = df_price[10:10+N_train]
train_dr = df_y[10:10+N_train]*100

history = model.fit(train_price,train_dr, validation_data = (test_price, test_dr), epochs=500, batch_size=128, verbose=0)

In [43]:

fig = go.Figure()
fig.add_trace(go.Scatter(y=np.array(history2.history['loss'])/10000,name='MLP_train'))
fig.add_trace(go.Scatter(y=np.array(history2.history['val_loss'])/10000,name='MLP_val'))
fig.add_trace(go.Scatter(y=np.array(history.history['loss'])/10000,name='RNN_train'))
fig.add_trace(go.Scatter(y=np.array(history.history['val_loss'])/10000,name='RNN_val'))
fig.update_layout(width=700,height=400)
fig.show()

In [58]:
i = np.random.randint(1)
dr3 = model(test_price[i:i+1])
t1 = go.Scatter(x=np.arange(24), y=test_price[i],mode='lines', name='price',line = {'width':3,'dash':'dash','color':'#ffa05a'},yaxis='y2')
t2 = go.Scatter(x=np.arange(24), y=test_dr[i],mode='lines', name='True',line = {'width':6,'dash':'dash'})
#t3 = go.Scatter(x=np.arange(24), y=dr1[0],mode='lines', name='RNN(300)',line = {'width':3})
t6 = go.Scatter(x=np.arange(24), y=dr3[0],mode='lines', name='MLP',line = {'width':3})
fig = go.Figure(data=[t6,t2],layout={
    'xaxis':{'title':'Hour', 'showline':True},
    'yaxis':{'title': 'Dispatch(MW)'},
    'yaxis2': {'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[0,80], 'showgrid':False},
    "font": {"size": 40,'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':1600,
    'height':900
})
fig.show()
